In [0]:
%pip install pytest==8.4.2

## ⚠️ Important Notice: Use a Personal Cluster for Structured Streaming

**Attention students:**  

This notebook uses **Structured Streaming** features that are **not fully supported in serverless clusters**.  

To ensure full functionality, including:   
- Continuous streaming processing  
- Checkpointing and state management  

You **must use a personal (interactive) cluster** or a standard Databricks cluster with the correct runtime.  

> ❌ Serverless clusters may limit streaming capabilities, prevent proper checkpointing, and can cause runtime errors.  

**Recommended actions:**  
1. Start a **personal cluster** in your workspace.  
2. Make sure the cluster uses a **Databricks Runtime version that supports Structured Streaming** (preferably latest LTS).  
3. Attach this notebook to the personal cluster before running or creating your DLT pipeline.  

Following this ensures your streaming pipelines work correctly and all transformations execute as expected.


In [0]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.streaming import StreamingQuery
from helpers import utils, generate_streaming_events as gen_stream

## 🗂️ Defining Paths and Environment Variables

In this step, we will **set up the environment and directory structure** that will be used during the streaming process.

- `capstone_env`: defines the environment (e.g., `dev`, `prod`), making the code reusable across different stages.  
- `base_schema`: retrieves the base schema name using the helper function `utils.get_base_user_schema()`.  
- `volume_base_path`: base directory path where all data and metadata related to the stream will be stored.  
- `raw_path`: location where the raw event files will be written.  
- `checkpoint_path`: location where Spark will save checkpoints to keep track of stream progress.  
- `schema_location_path`: directory where the evolving schema will be stored.  
- `output_table`: name of the Delta table where the processed events will be written.

We also print the paths to verify that they were set correctly.


In [0]:
from instructors.src.solutions.streaming_ingestion import StreamingIngestion

mod2 = StreamingIngestion()
catalog = utils.get_param("catalog", "capstone_dev")
env = utils.get_param("env", "dev")
bronze_schema = f"{utils.get_base_user_schema()}_bronze"

volume_base_path = f"/Volumes/{catalog}/{bronze_schema}"
raw_path = f"{volume_base_path}/raw_files/web_site_events"
checkpoint_path = f"{volume_base_path}/checkpoint_files/web_site_events"
schema_location_path = f"{volume_base_path}/schema_files/web_site_events"
output_table = f"{catalog}.{bronze_schema}.web_site_events"

print(f"RAW PATH: {raw_path}")
print(f"CHECKPOINT: {checkpoint_path}")
print(f"SCHEMA LOCATION: {schema_location_path}")
print(f"TABLE: {output_table}")

## 📥 Defining a Function to Read Streaming Data (JSON)

In this step, we define a helper function called `read_stream_json` to **read JSON files in streaming mode** using **Auto Loader**.  

This function allows Spark to automatically detect and adapt to schema changes by enabling **schema evolution**.

### Function details:
- `spark`: the active `SparkSession`.
- `source_path`: the path to the directory containing the raw JSON files.
- `schema_path`: the path where Spark will store the inferred schema to track changes over time.

### Key options:
- `cloudFiles.format = "json"` → defines the file format.  
- `cloudFiles.inferColumnTypes = true` → automatically detects column types.  
- `cloudFiles.schemaEvolutionMode = addNewColumns` → allows adding new columns without failing the stream.  
- `cloudFiles.schemaLocation` → directory to store schema information.  
- `multiLine = true` → supports JSON files spanning multiple lines.


In [0]:
def read_stream_json(self, source_path: str, schema_path: str) -> DataFrame:
    """
    This function returns a streaming DataFrame that can be used for further transformations or writes.
    Args:
        source_path (str): Path to the directory containing the source JSON files.
        schema_path (str): Path to the schema location used by Auto Loader to track and evolve the schema.
    Returns:
        DataFrame: A streaming DataFrame representing the ingested JSON data.
    """
    pass

## 💾 Defining a Function to Write Streaming Data to Delta Lake

In this step, we define the helper function `write_stream_to_delta` to **write streaming data to a Delta Lake table** with **schema evolution enabled** through the `mergeSchema` option.

### Function details:
- `df`: the streaming DataFrame that will be written to Delta Lake.  
- `checkpoint_path`: the directory where Spark will store checkpoints to maintain stream state.  
- `table_name`: the name of the target Delta table.

### Key options:
- `format("delta")` → writes the stream in Delta Lake format.  
- `checkpointLocation` → stores metadata and progress information for fault tolerance.  
- `mergeSchema = true` → allows Delta Lake to automatically update the table schema as new columns appear.  
- `outputMode = append` → continuously appends new records to the table.  
- `trigger(availableNow=True)` → processes all available data and then stops (useful for development of streaming pipelines).


In [0]:
def write_stream_to_delta(self, df: DataFrame, checkpoint_path: str, table_name: str) -> StreamingQuery:
    """
    This function returns a `StreamingQuery` object that represents the active streaming job.
    Args:
        df (DataFrame): The streaming DataFrame to be written to Delta Lake.
        checkpoint_path (str): Path to the directory for storing checkpoint data.
        table_name (str): Name of the target Delta Lake table.
    Returns:
        StreamingQuery: The active streaming query writing data to Delta Lake.
    """
    pass

## 🚀 Running the Streaming Job with the Initial Schema

In this step, we will **execute the streaming pipeline using the initial schema** (without schema evolution enabled).

### Instructions:
1. **Open the simulation script** at: helpers/generate_streaming_events.py

2. **Run the script with the parameter**:
```bash
enable_schema_evolution=False
```

This ensures that schema evolution is **disabled**, so the stream will not create new columns.

*(Optional)* You can adjust the following parameters in the script to test different scenarios:

- `batch_size`: number of events per batch
- `delay_seconds`: delay between batches
- `total_batches`: total number of batches
- `null_frequency`: frequency of null values in the data

### What this code does:

- Retrieves or creates the active `SparkSession`.
- Reads streaming JSON data from `raw_path` using the `read_stream_json` function.
- Prints the schema inferred from the stream.
- Writes the streaming data to a Delta table using `write_stream_to_delta` with checkpointing.
- Waits for the streaming query to process all available data.




In [0]:
gen_stream.stream_events(
    batch_size=10000,
    delay_seconds=1,
    total_batches=10,
    null_frequency=500,
    env=env,
    enable_schema_evolution=False
)

In [0]:
df_stream = mod2.read_stream_json(raw_path, schema_location_path)
df_stream.printSchema()

query = mod2.write_stream_to_delta(df_stream, checkpoint_path, output_table)
query.awaitTermination()

## ✅ Validating the Delta Table

After running the streaming job, it's important to **verify that the data was correctly written** to the Delta table.

The following command:

- Executes a simple SQL query on the Delta table defined by `output_table`.  
- Retrieves the first 100 records to give a quick preview of the data.  
- Displays the results in a tabular format for easy inspection.


In [0]:
spark.sql(f"SELECT * FROM {output_table} limit 100").display()
spark.sql(f"SELECT COUNT(*) FROM {output_table}").display()

%md
## 🚀 Running the Streaming Job with Schema Evolution Enabled

In this step, we will **execute the streaming pipeline with schema evolution enabled**.  
This allows Spark Structured Streaming to **automatically detect and incorporate new columns** into the Delta table schema as they appear in the incoming data.

### Instructions:
1. **Open the simulation script** at: `helpers/generate_streaming_events.py`

2. **Run the script with the parameter**:
```bash
enable_schema_evolution=True
```
This enables Spark to **adapt to schema changes** during the streaming process.

*(Optional)* You can adjust the same simulation parameters as before to test different scenarios:

- `batch_size`: number of events per batch
- `delay_seconds`: delay between batches
- `total_batches`: total number of batches
- `null_frequency`: frequency of null values in the data

### What this code does:

- Retrieves or creates the active `SparkSession`.
- Reads streaming JSON data from `raw_path` using the `read_stream_json` function with **schema evolution support**.
- Prints the schema inferred from the stream, showing any new columns added.
- Writes the streaming data to a Delta table using `write_stream_to_delta` with **mergeSchema enabled**.
- Waits for the streaming query to process all available data.


In [0]:
gen_stream.stream_events(
    batch_size=10000,
    delay_seconds=1,
    total_batches=10,
    null_frequency=500,
    env=env,
    enable_schema_evolution=True
)

In [0]:
df_stream = mod2.read_stream_json(raw_path, schema_location_path)
df_stream.printSchema()

query = mod2.write_stream_to_delta(df_stream, checkpoint_path, output_table)
query.awaitTermination()

### Did the read/write fail?
> **Note:**  
> Remember the default behavior for autoloader in mode `addNewColumns` is:
>
> - Stream fails. 
> - New columns are added to the schema. 
> - Existing columns do not evolve data types.  
>
> We need to **rerun** the read/write operation to see the changes reflected on the destination table.  
>
> Checking at the schema folder / volume, you should be able to see two different versions.  
>
> See more: https://docs.databricks.com/aws/en/ingestion/cloud-object-storage/auto-loader/schema#how-does-auto-loader-schema-evolution-work

## ✅ Validating the Delta Table After Schema Evolution

After running the streaming job with **schema evolution enabled**, it's important to **verify that new columns were correctly added** to the Delta table.

The following command:

- Executes a SQL query on the Delta table defined by `output_table`.  
- Filters for records where the newly added column `referrer_url` is not null, helping to confirm that schema evolution worked.  
- Retrieves the first 20 records to quickly inspect the updated schema and data.  
- Displays the results in a tabular format for easy observation of the new column.


In [0]:
spark.sql(f"SELECT * FROM {output_table} where referrer_url is not null limit 20").display()
spark.sql(f"SELECT COUNT(*) FROM {output_table}").display()

## Tests
**TO DO:**  
- Check if there are any failed tests and investigate their root cause

In [0]:
from helpers import test_runner
import os

notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
os.environ["NOTEBOOK_NAME"] = notebook_path.split("/")[-1]

test_runner.run(
    df=spark.sql(f"SELECT * FROM {output_table}"),
    schema_evoluted=True
)